In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


from astropy.io import fits
import dill as pickle
from emcee import EnsembleSampler
import numpy as np
import os
import sys

pwd = os.getcwd()
little_things_root_dir = os.path.dirname(pwd)
sys.path.append(little_things_root_dir)

from little_things_lib.galaxy_piecewise import Galaxy
from little_things_lib.piecewise_mcmc_fitter import (
    EmceeParameters,
    generate_nwalkers_start_points, 
    lnprob)
from little_things_lib.plotting import plot_posterior_distributions, plot_walker_paths
from datetime import datetime


RAD_PER_ARCSEC = np.pi / (60*60*180)

/home/rohit/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

## Enter parameters for galaxy in cell below

In [3]:
galaxy_name = 'NGC2366'

velocity_error_2d = 20  # [km/s]

DEG_PER_PIXEL = 4.17e-4
DISTANCE = 3400 # kpc

# can leave these as any number for now, not used
LUMINOSITY = 1e8  # solar luminositiy
HI_MASS = 1e8  # solar masses

In [4]:
mcmc_params = EmceeParameters(
    ndim=3,
    nwalkers=10, 
    nburn=100,
    niter=300,
    nthin=3,
    nthreads=4
)


In [5]:
"""
Expect the data to be provided in following naming convention in 'data' directory:

2D observed velocity field FITS file: <galaxy_name>_1mom.fits
Bbarolo fit parameters text file: <galaxy_name>_ring_parameters.txt
Stellar velocity curve: <galaxy_name>_stellar_velocities.txt
Gas velocity_curve: <galaxy_name>_gas_velocities.txt

"""


data_dir = os.path.join(little_things_root_dir, 'data')

observed_2d_vel_field_fits_file = os.path.join(data_dir, f'{galaxy_name}_1mom.fits')
ring_parameters_file = os.path.join(data_dir, f'{galaxy_name}_ring_parameters.txt')

stellar_velocities_file = os.path.join(data_dir, f'{galaxy_name}_stellar_velocities.txt')
gas_velocities_file = os.path.join(data_dir, f'{galaxy_name}_gas_velocities.txt')


In [6]:
radii_arcsec, test_rotation_curve, inclinations, position_angles, x_centers, y_centers , v_systemics = \
    np.loadtxt(ring_parameters_file, usecols=(1,2,4,5,-4,-3,-2)).T

radii_kpc = radii_arcsec * RAD_PER_ARCSEC * DISTANCE
avg_inclination = np.mean(inclinations)
avg_position_angle = np.mean(position_angles)
avg_x_center = np.mean(x_centers)
avg_y_center = np.mean(y_centers)
v_systemic = np.mean(v_systemics)

observed_2d_vel_field = fits.open(observed_2d_vel_field_fits_file)[0].data

stellar_radii, stellar_vel = np.loadtxt(stellar_velocities_file, unpack=True)
gas_radii, gas_vel = np.loadtxt(gas_velocities_file, unpack=True)

# comment out below and uncomment above lines when actually doing a fit
# the lines below are for testing only!
#stellar_radii = np.linspace(0, 7, 41)
#gas_radii = np.linspace(0, 7, 41)
#stellar_vel = np.linspace(0, 1, 41)
#gas_vel = np.linspace(0, 1, 41)

In [7]:
nparams=10
bin_edges=[]
indices=np.linspace(0,len(radii_arcsec)-1,nparams)
#np.linspace/np.logpsace
for i in indices:
    bin_edges.append(radii_arcsec[int(i)])

In [8]:
galaxy = Galaxy(
    distance_to_galaxy=DISTANCE,  # [kpc] Look this up for the galaxy 
    deg_per_pixel=DEG_PER_PIXEL ,  # 'CRDELT1' and 'CRDELT2' in the FITS file header (use absolute value)
    bin_edges=bin_edges,
    galaxy_name=galaxy_name,
    vlos_2d_data=observed_2d_vel_field,
    v_error_2d=velocity_error_2d,
    output_dir='output',
    luminosity=LUMINOSITY,
    HI_mass=HI_MASS)

tilted_ring_params = {
    'v_systemic': v_systemic,
    'radii': radii_kpc,   
    'inclination': inclinations,
    'position_angle': position_angles,
    'x_pix_center': x_centers,
    'y_pix_center': y_centers
}

galaxy.set_tilted_ring_parameters(**tilted_ring_params)

galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='stellar',
    rotation_curve_radii=stellar_radii,
    rotation_curve_velocities=stellar_vel)

galaxy.interpolate_baryonic_rotation_curve(
    baryon_type='gas',
    rotation_curve_radii=gas_radii,
    rotation_curve_velocities=gas_vel)



### The cell below should be modified for the piecewise model

In [9]:

# initialize MCMC start position and bounds
galaxy.set_piecewise_prior_bounds(
    
    bin_edges
  
)

start_pos = generate_nwalkers_start_points(

    mcmc_params.nwalkers,
    galaxy
)

In [10]:
# initialize sampler
sampler = EnsembleSampler(
    mcmc_params.nwalkers,
    mcmc_params.ndim, 
    lnprob, 
    args=[galaxy], 
    threads=mcmc_params.nthreads)

# burn in 
sampler.run_mcmc(start_pos, N=mcmc_params.nburn)
start_pos_after_burn_in = sampler._last_run_mcmc_result[0]
sampler.reset()

emcee: Exception while calling your likelihood function:
  params: [49.97002294 33.83617204 59.12609227 43.70672178 44.51856367 24.89817985
  4.70640751 58.6886733  43.59167759 28.72616176]
  args: [<little_things_lib.galaxy_piecewise.Galaxy object at 0x7f302f9584d0>]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/rohit/anaconda3/lib/python3.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/rohit/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 83, in lnprob
    lnl, bb = lnlike(params, galaxy)
  File "/home/rohit/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 56, in lnlike
    assert v_m.shape==galaxy.bin_edges.shape
AttributeError: 'list' object has no attribute 'shape'


emcee: Exception while calling your likelihood function:
  params: [49.08022043  2.94182978 28.78905318  4.89692769 40.38470665  6.65263541
 48.78689479 16.3642273  31.95853884 17.7891229 ]
  args: [<little_things_lib.galaxy_piecewise.Galaxy object at 0x7f302f958090>]
  kwargs: {}
  exception:


Traceback (most recent call last):


emcee: Exception while calling your likelihood function:


...

  kwargs: {}


  File "/home/rohit/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 83, in lnprob
    lnl, bb = lnlike(params, galaxy)
  File "/home/rohit/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 56, in lnlike
    assert v_m.shape==galaxy.bin_edges.shape


  exception:


Traceback (most recent call last):
AttributeError: 'list' object has no attribute 'shape'
  File "/home/rohit/anaconda3/lib/python3.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/rohit/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 83, in lnprob
    lnl, bb = lnlike(params, galaxy)
  File "/home/rohit/little-things/little_things_lib/piecewise_mcmc_fitter.py", line 56, in lnlike
    assert v_m.shape==galaxy.bin_edges.shape
AttributeError: 'list' object has no attribute 'shape'


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
sampler = EnsembleSampler(
    mcmc_params.nwalkers,
    mcmc_params.ndim, 
    lnprob, 
    args=[galaxy], 
    threads=mcmc_params.nthreads)

# this will break up the fitting procedure into smaller chunks of size batch_size and save progress
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y")

batch_size = 4
mcmc_output = []
for batch in range(mcmc_params.niter // batch_size):
    if batch == 0:
        batch_start = start_pos_after_burn_in
    else:
        batch_start = None
        sampler.pool = temp_pool
    mcmc_output += sampler.run_mcmc(batch_start, batch_size, thin=mcmc_params.nthin)
    temp_pool = sampler.pool
    del sampler.pool
    with open(f'sampler_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(sampler, f)
    with open(f'mcmc_output_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(mcmc_output, f)
    print(f"Done with steps {batch*batch_size} - {(batch+1)*batch_size} out of {mcmc_params.niter}")


#### The two cells below demonstrate how to load your saved results. This is useful in the follow cases:

1) You finished a long MCMC fitting run and want to plot the results without having to redo the whole thing.

2) Your computer crashed before it was done running the previous MCMC fit and you want to restart it from the last saved iteration. In this case you can run the second cell below to continue the MCMC fitting. The results of the restarted run will be saved separately from the results of the previous run. The combined results will also be saved.

In [ ]:
# example of how to load the pickled objects
# change the name of the files to whatever yours were saved as

with open('sampler_09-Apr-2020.pkl', 'rb') as f:
    saved_sampler = pickle.load(f)
    restart_pos = saved_sampler._last_run_mcmc_result[0]
with open('mcmc_output_09-Apr-2020.pkl', 'rb') as f:
    saved_mcmc_output = pickle.load(f)

In [ ]:
# example of how to restart the MCMC fit from the last save point 
# assumes you loaded the sampler and mcmc_output from the saved pickles in the example above

restart_sampler = EnsembleSampler(
    mcmc_params.nwalkers,
    mcmc_params.ndim, 
    lnprob, 
    args=[galaxy], 
    threads=mcmc_params.nthreads)
restart_mcmc_output = []

dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y")

batch_size = 10
for batch in range(mcmc_params.niter // batch_size):
    if batch==0:
        batch_start = restart_pos
    else:
        batch_start = None
        restart_sampler.pool = temp_pool
    restart_mcmc_output += restart_sampler.run_mcmc(batch_start, batch_size, thin=mcmc_params.nthin)
    temp_pool = restart_sampler.pool
    del sampler.pool
    with open(f'sampler_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(restart_sampler, f)
    with open(f'mcmc_output_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(restart_mcmc_output, f)
    print(f"Done with steps {batch*batch_size} - {(batch+1)*batch_size} out of {mcmc_params.niter}")

    
# this step adds the MCMC results from the restarted run to the ones that were saved from the previous run.
total_mcmc_output = saved_mcmc_output + restart_mcmc_output
    with open(f'total_mcmc_output_{timestampStr}.pkl', 'wb') as f:
        pickle.dump(total_mcmc_output, f)

In [ ]:
#### Plot the posterior distributions and walker paths

# Uncomment below line if using the the sampler loaded from the save file
#sampler=saved_sampler

parameter_labels = sorted(list(galaxy.bounds.keys()))

plot_posterior_distributions(sampler, labels=parameter_labels)

plot_walker_paths(
    sampler,
    mcmc_params,
    labels=parameter_labels)

def get_fit_stats(sampler, labels=parameter_labels):
    for i, label in enumerate(labels):
        chain = sampler.chain[:, :, i].flatten()
        print(f"{label}: {np.mean(chain)} +/- {np.std(chain)}")
        
get_fit_stats(sampler)

In [ ]:
#blobs = np.reshape(sampler.blobs, (int(mcmc_params.niter/mcmc_params.nthin) * mcmc_params.nwalkers, 10))
blobs = [] 
for subarr in sampler.blobs:
    blobs += subarr
blobs = np.array(blobs)

rotcurve_index = len(galaxy.bounds) + 1
v_tot = np.mean( blobs[:, rotcurve_index])

plt.plot(radii_kpc, test_rotation_curve, linewidth=2., label="data", color="black")

plt.step(galaxy.bin_edges[:-1], v_tot, where="post", color=black)
plt.plot(galaxy.bin_edges[-2:], [v_tot[-1], v_tot[-1], color=black], label="total, model")

plt.xlabel("r [kpc]")
plt.ylabel("v [km/s]")
plt.legend()